In [1]:
import sqlite3
from urllib.request import urlopen as urlReq
from bs4 import BeautifulSoup as bSoup
from selenium import webdriver

#function for calculating discount price
def calculate_discount(current_price, old_price):
    if old_price: #if old price exists, it means there is a discount
        a = float(current_price)
        b = float (old_price)
    
        discount = 100 * (b - a) / b
    
        return discount
    else:
        return 0
 
# i filtered Euronics's keyboard page with conditions that it had to be mechanical and 100% size

myUrl = 'https://www.euronics.ee/it/lisaseadmed/klaviatuurid?f=IgYIzREQg2ciBgjPERCFZw'

driver = webdriver.Chrome()

#reading the contents of the page
#this only gets the products that are on the first page
urlConnection = urlReq(myUrl)
content = urlConnection.read()
urlConnection.close()

#parsing the conted using BeautifulSoup
parsedContent = bSoup(content, "html.parser")

#find all products
products = parsedContent.findAll("article", {"class":"product-card vertical"})

#creating a database for information (if not created yet) and connecting to it
connection = sqlite3.connect('keyboards.db')
c = connection.cursor()

#i want to get the product name, current price, old price and link to product from the website and calculate the discount
#creating a table for the information i am saving
c.execute('''
          CREATE TABLE IF NOT EXISTS keyboards
          (product_name TEXT,
          current_price TEXT,
          old_price TEXT,
          discount REAL,
          link_to_product TEXT)
          ''')

connection.commit()    

#exctract name and price data from content and output it to db file
for product in products:
    #seperating product name
    name_container = product.findAll("span", {"class":"product-card__title"})
    product_name = name_container[0].text.strip()
    
    #separating current and old price
    price_container = product.findAll("div", {"class":"price"})
    price = " ".join(price_container[0].text.split())
    pricesList = price.replace(",", " ").split("€")
    
    current_price = ".".join(pricesList[0].split())
    old_price = ".".join(pricesList[1].split())
    
    #separating link to product
    link_container = product.findAll("a", {"class":"product_name"})
    link_to_product = link_container[0].get("href")
    
    #calculate discount
    discount = int(calculate_discount(current_price, old_price))
    
    #adding information to database
    c.execute("""INSERT INTO keyboards VALUES (?, ?, ?, ?, ?)""",
              (product_name, current_price, old_price, discount, link_to_product))
    
    connection.commit()
    
c.close()

print("Data saved.")

Data saved.
